In [ ]:
import pandas as pd
df=pd.read_csv('../data/raw/IMDB_four_genre_larger_plot_description.csv')
df.drop('movie_id',axis=1,inplace=True)
df.head()

,description,genre
0,Elle Evans (Joey King) has finally completed h...,romance
1,A young girl tries to understand how she myste...,horror
2,"In 1800s England, a well meaning but selfish y...",comedy
3,Abby Holland (Kristen Stewart) and Harper Cald...,romance
4,Olga and Maks are 15 years apart. She is a suc...,romance
